#book recommender system using collaborative clustering


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [3]:
# books=pd.read_csv("BX-Books.csv",sep=';',encoding='latin-1',error_bad_lines=False) 
with open("BX-Books.csv", "r", encoding="latin-1") as f:
    reader = csv.reader(f, delimiter=";")

FileNotFoundError: [Errno 2] No such file or directory: 'BX-Books.csv'

In [ ]:
books=books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

In [ ]:
books.rename(columns={
    "Book-Title":"title",
    "Books-Author":"author",
    "Year-Of-Publication":"year",
    "Image-URL-L":"img_url"
},inplace=True)

In [ ]:
users=pd.read_csv("BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-173-9ca8b7c8fe35>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users=pd.read_csv("sample_data/BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In [ ]:
ratings=pd.read_csv("BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-174-252b9ded5838>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings=pd.read_csv("sample_data/BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In collaborative system it is important for a user to give you ratings.

In [ ]:
ratings.rename(columns={"User-ID":"user_id","Book-Rating":"rating"},inplace=True)

In [ ]:
x=ratings["user_id"].value_counts()>200

In [ ]:
y=x[x].index

In [ ]:
ratings=ratings[ratings['user_id'].isin(y)]

In [ ]:
ratings_with_books=ratings.merge(books,on="ISBN")

In [ ]:
num_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [ ]:
num_rating.rename(columns={"rating":"num_of_rating"},inplace=True)

In [ ]:
final_rating=ratings_with_books.merge(num_rating,on ='title')

In [ ]:
final_rating=final_rating[final_rating["num_of_rating"]>=50]

In [ ]:
final_rating.sample(10)

,user_id,ISBN,rating,title,Book-Author,year,Publisher,num_of_rating
10354,46398,0553280368,0,C Is for Corpse (Kinsey Millhone Mysteries (Pa...,Sue Grafton,1987,Bantam,93
80170,239584,1565117794,0,Life of Pi,Yann Martel,2003,Highbridge Audio,185
18641,166123,0425147363,0,Tom Clancy's Op-Center (Tom Clancy's Op Center...,Tom Clancy,1995,Berkley Publishing Group,63
82104,8245,1569471169,7,Falling Angels,Barbara Gowdy,1998,Soho Press,61
150812,68555,0060959037,0,Prodigal Summer: A Novel,Barbara Kingsolver,2001,Perennial,50
2046,108352,0316602051,0,The Jester,James Patterson,2003,"Little, Brown",68
8650,143415,0446611212,0,Violets Are Blue,James Patterson,2002,Warner Vision,150
20392,198711,0451162072,0,Pet Sematary,Stephen King,1994,Signet Book,119
42732,87555,0553272535,0,Night,Elie Wiesel,1982,Bantam Books,76
25197,129358,0345313860,0,"The Vampire Lestat (Vampire Chronicles, Book II)",ANNE RICE,1986,Ballantine Books,123


In [ ]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [ ]:
book_pivot=final_rating.pivot_table(columns='user_id',index='title',values='rating')

In [ ]:
book_pivot.fillna(0,inplace=True)

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
book_sparse=csr_matrix(book_pivot)

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
model=NearestNeighbors(algorithm='brute')


In [ ]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
distance,suggestion=model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)


In [ ]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [ ]:
suggestion

array([[237, 240, 238, 241, 184, 536]])

In [ ]:
for i in range(len(suggestion)):
  print(book_pivot.index[suggestion[i]])
  #to see the recommendation

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [ ]:
book_pivot.index[3]

'4 Blondes'

In [ ]:
books_name=book_pivot.index

In [ ]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(books_name,open('artifacts/mbook_name.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))


In [ ]:
def recommend_me_the_book(book_name):
  try:
    book_id=np.where(book_pivot.index ==book_name)[0][0]
    distance,suggestion=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
  
    for i in range(len(suggestion)):
      books=book_pivot.index[suggestion[i]]
      for j in books:
        print(j)
  except IndexError:
    print("The book doesnt exists in the database")
  


In [ ]:
book_name=''
recommend_me_the_book(book_name)

The book doesnt exists in the database
